In [1]:
import tensorflow as tf

In [2]:
MAX_LEN = 50
SOS_ID = 1

In [3]:
def make_dataset(file_path):
    dataset = tf.data.TextLineDataset(file_path)
    dataset = dataset.map(lambda string: tf.string_split([string]).values)
    # 将字符串形式的单词编号-> 整数
    dataset = dataset.map(lambda string: tf.string_to_number(string, tf.int32))
    dataset = dataset.map(lambda x: (x, tf.size(x)))
    return dataset

In [4]:
# 从源语言文件 src_path 和目标语言 文件 trg_path 分别读取数据，并进行填充和batching操作
def make_src_trg_dataset(src_path, trg_path, batch_size):
    src_data = make_dataset(src_path)
    trg_data = make_dataset(trg_path)
    # zip 合并后 每项ds 由4个张量组成
    #  ds[0][0] 是源句子
    #  ds[0][1] 是源句子长度
    #  ds[1][0] 是目标句子
    #  ds[1][1] 是目标句子长度
    
    # 处理内容为空和长度过长的句子
    def filter_length(src_tuple, trg_tuple):
        ((src_input, src_len), (trg_label, trg_len)) = (src_tuple, trg_tuple)
        src_len_ok = tf.logical_and(
            tf.greater(src_len, l), tf.less_equal(src_len, MAX_LEN))
        trg_len_ok = tf.logical_and(
            tf.greater(trg_len, l), tf.less_equal(trg_len, MAX_LEN))
        return tf.logical_and(src_len_ok, trg_len_ok)
    dataset = dataset.filter(filter_length)
    # 解码器需要两种格式的的目标句子：
    #  1.解码器的输入（trg_input），形式如同：“<sos X Y Z>”
    #  2.解码器的目标输出（trg_label），形式如同“X Y Z <eos>”
    # 从文件中读到的是“<sos X Y Z>” 形式，我们需要从中生成“X Y Z <eos>”形式并加入到Dataset中
    def make_trg_input(src_tuple, trg_tuple):
        ((src_input, src_len), (trg_label, trg_len)) = (src_tuple, trg_tuple)
        trg_input = tf.concat([[SOS_ID], trg_label[:-1]], axis=0)
        return ((src_input, src_len), (trg_input, trg_label, trg_len))
    dataset = dataset.map(make_trg_input)
    
    # 随机打乱训练数据
    dataset = dataset.shuffle(10000)
    
    # 规定填充后的输出的数据维度。
    padded_shapes = (
        (tf.TensorShape([None]),   # 源句子是长度未知向量
         tf.TensorShape([])),      # 源句子长度是单个数字
        (tf.TensorShape([None]),   # 目标句子（解码器输入）是长度未知的向量
         tf.TensorShape([None]),   # 目标句子（解码器目标输出）是长度未知的向量
         tf.TensorShape([])))      # 目标句子长度是单个数字
    # 调用padded_batch 方法进行batching操作
    batched_dataset = dataset.padded_batch(batch_size, padded_shapes)
    return batched_dataset